<a href="https://colab.research.google.com/github/ErnestoCash/ASAP/blob/main/MedChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk tensorflow
import nltk
import numpy as np
import random
import json
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder

nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
intents = {
    "intents": [
        {"tag": "make_appointment",
         "patterns": ["I want to schedule an appointment", "Book an appointment", "Need to make a new appointment"],
         "responses": ["Sure, I can help you with that. What date and time are you looking at?"]},
        {"tag": "cancel_appointment",
         "patterns": ["I need to cancel my appointment", "Cancel my booking", "I can't make it to the appointment"],
         "responses": ["No problem, I've canceled your appointment. Would you like to reschedule?"]},
        {"tag": "get_address",
         "patterns": ["What's the hospital address?", "Where are you located?", "I need the hospital's address"],
         "responses": ["We are located at 123 Health St., Wellness City."]}
    ]
}


In [3]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []

# Loop through each intent and tokenize each pattern
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ['?', '.', '!']]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))


In [4]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffle and convert to array
random.shuffle(training)
training = np.array(training, dtype=object)

# Split features and labels
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))


In [5]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)


Epoch 1/200
2/2 [==============================] - 1s 13ms/step - loss: 1.0743 - accuracy: 0.5556
Epoch 2/200
2/2 [==============================] - 0s 8ms/step - loss: 1.0858 - accuracy: 0.3333
Epoch 3/200
2/2 [==============================] - 0s 7ms/step - loss: 1.0101 - accuracy: 0.5556
Epoch 4/200
2/2 [==============================] - 0s 9ms/step - loss: 1.1277 - accuracy: 0.4444
Epoch 5/200
2/2 [==============================] - 0s 9ms/step - loss: 1.0657 - accuracy: 0.4444
Epoch 6/200
2/2 [==============================] - 0s 8ms/step - loss: 0.7993 - accuracy: 0.7778
Epoch 7/200
2/2 [==============================] - 0s 10ms/step - loss: 1.0187 - accuracy: 0.4444
Epoch 8/200
2/2 [==============================] - 0s 8ms/step - loss: 0.9019 - accuracy: 0.6667
Epoch 9/200
2/2 [==============================] - 0s 8ms/step - loss: 0.8588 - accuracy: 0.6667
Epoch 10/200
2/2 [==============================] - 0s 9ms/step - loss: 0.9413 - accuracy: 0.6667
Epoch 11/200
2/2 [=========

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print("Chatbot is running!")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)


Chatbot is running!
